In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

fpath_fasta = 'current_Bacteria_unaligned.fa'
seq_data = open(fpath_fasta, 'r').read()
lines = seq_data.split('\n')

In [17]:
def get_seq_dict(desc, seq):
    try:
        keys = ['domain', 'phylum', 'class', 'subclass', 'order', 'suborder', 'family', 'genus']
        txn_info = desc.split('rootrank;')[1].split(';')
        seq_info = {}
        for i, key in enumerate(keys):
            try:
                idx = txn_info.index(key)
                seq_info[key] = txn_info[idx-1]
            except ValueError:
                seq_info[key] = None

        seq_info['seq_len'] = len(seq)
        seq_info['sequence'] = seq
    except Exception as e:
        print(i)
        print(txn_info)
        print(desc)
        raise IndexError(e)
    
    return seq_info

In [18]:
desc = ''
seq = ''

seq_dicts = []

for i, line in enumerate(tqdm(lines, total=len(lines))):
    if line.startswith('>'):
        if len(desc) > 0 and len(seq) > 0:
            seq_dicts.append(get_seq_dict(desc, seq))
        desc = line
        seq = ''
    else:
        seq += line

In [19]:
df_seqs = pd.DataFrame(seq_dicts)
df_seqs

,class,domain,family,genus,order,phylum,seq_len,sequence,subclass,suborder
0,Actinobacteria,Bacteria,Acidimicrobiaceae,Acidimicrobium,Acidimicrobiales,"""Actinobacteria""",454,gcggcgtgctacacatgcagtcgtacgcggtggcacaccgagtggc...,Acidimicrobidae,"""Acidimicrobineae"""
1,Actinobacteria,Bacteria,Acidimicrobiaceae,Acidimicrobium,Acidimicrobiales,"""Actinobacteria""",1329,gacgaacgctggcggcgtgcctaacacatgcaagtcgtacgcggtg...,Acidimicrobidae,"""Acidimicrobineae"""
2,Actinobacteria,Bacteria,Acidimicrobiaceae,Acidimicrobium,Acidimicrobiales,"""Actinobacteria""",1122,accccgaagtgggggataacaccgggaaaccggtgctaataccgca...,Acidimicrobidae,"""Acidimicrobineae"""
3,Actinobacteria,Bacteria,Acidimicrobiaceae,Acidimicrobium,Acidimicrobiales,"""Actinobacteria""",1365,tngcgaacgggtgcgtaacacgtagggaacccaccccgacgtgggg...,Acidimicrobidae,"""Acidimicrobineae"""
4,Actinobacteria,Bacteria,Acidimicrobiaceae,Acidimicrobium,Acidimicrobiales,"""Actinobacteria""",1044,tgagacacggcccagactctacgggaggcagcagtggggatattgc...,Acidimicrobidae,"""Acidimicrobineae"""
5,Actinobacteria,Bacteria,Acidimicrobiaceae,Acidimicrobium,Acidimicrobiales,"""Actinobacteria""",1262,aaatactntcccccggnaaccgnanngtccagggagcaaagcctcc...,Acidimicrobidae,"""Acidimicrobineae"""
6,Actinobacteria,Bacteria,Acidimicrobiaceae,Acidimicrobium,Acidimicrobiales,"""Actinobacteria""",461,gacgctgcggcntgcctaccatgcaagtcctacgcggtggcacacc...,Acidimicrobidae,"""Acidimicrobineae"""
7,Actinobacteria,Bacteria,Acidimicrobiaceae,Acidimicrobium,Acidimicrobiales,"""Actinobacteria""",458,atgcggcgtgctagcacatgcagtcgtacgcggtggcaacaccgag...,Acidimicrobidae,"""Acidimicrobineae"""
8,Actinobacteria,Bacteria,Acidimicrobiaceae,Acidimicrobium,Acidimicrobiales,"""Actinobacteria""",1329,gacgaacgctggcggcgtgcctaacacatgcaagtcgtacgcggtg...,Acidimicrobidae,"""Acidimicrobineae"""
9,Actinobacteria,Bacteria,Acidimicrobiaceae,Acidimicrobium,Acidimicrobiales,"""Actinobacteria""",1329,gacgaacgctggcggcgtgcctaacacatgcaagtcgtacgcggtg...,Acidimicrobidae,"""Acidimicrobineae"""


In [49]:
seq_lens = df_seqs['seq_len'].values

lens, counts = np.unique(seq_lens, return_counts=True)

print(np.sum(counts[np.where(np.logical_and(lens >= 1270, lens <= 1370))]))

552082


In [57]:
df_flen = df_seqs[df_seqs['seq_len'] >= 1270]
df_flen = df_flen[df_seqs['seq_len'] <= 1370]

df_flen

/home/srivathsa/miniconda3/envs/superpacs/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,class,domain,family,genus,order,phylum,seq_len,sequence,subclass,suborder
1,Actinobacteria,Bacteria,Acidimicrobiaceae,Acidimicrobium,Acidimicrobiales,"""Actinobacteria""",1329,gacgaacgctggcggcgtgcctaacacatgcaagtcgtacgcggtg...,Acidimicrobidae,"""Acidimicrobineae"""
3,Actinobacteria,Bacteria,Acidimicrobiaceae,Acidimicrobium,Acidimicrobiales,"""Actinobacteria""",1365,tngcgaacgggtgcgtaacacgtagggaacccaccccgacgtgggg...,Acidimicrobidae,"""Acidimicrobineae"""
8,Actinobacteria,Bacteria,Acidimicrobiaceae,Acidimicrobium,Acidimicrobiales,"""Actinobacteria""",1329,gacgaacgctggcggcgtgcctaacacatgcaagtcgtacgcggtg...,Acidimicrobidae,"""Acidimicrobineae"""
9,Actinobacteria,Bacteria,Acidimicrobiaceae,Acidimicrobium,Acidimicrobiales,"""Actinobacteria""",1329,gacgaacgctggcggcgtgcctaacacatgcaagtcgtacgcggtg...,Acidimicrobidae,"""Acidimicrobineae"""
10,Actinobacteria,Bacteria,Acidimicrobiaceae,Acidimicrobium,Acidimicrobiales,"""Actinobacteria""",1329,gacgaacgctggcggcgtgcctaacacatgcaagtcgtacgcggtg...,Acidimicrobidae,"""Acidimicrobineae"""
12,Actinobacteria,Bacteria,Acidimicrobiaceae,Acidimicrobium,Acidimicrobiales,"""Actinobacteria""",1333,gacgaacgctggcggcgtgcctaacacatgcaagtcgtacgcggtg...,Acidimicrobidae,"""Acidimicrobineae"""
13,Actinobacteria,Bacteria,Acidimicrobiaceae,Acidimicrobium,Acidimicrobiales,"""Actinobacteria""",1325,agagtttgatcatggctcaggacgaacgctggcggcgtgcctaaca...,Acidimicrobidae,"""Acidimicrobineae"""
14,Actinobacteria,Bacteria,Acidimicrobiaceae,Acidimicrobium,Acidimicrobiales,"""Actinobacteria""",1329,gacgaacgctggcggcgtgcctaacacatgcaagtcgtacgcggtg...,Acidimicrobidae,"""Acidimicrobineae"""
18,Actinobacteria,Bacteria,Acidimicrobiaceae,Acidimicrobium,Acidimicrobiales,"""Actinobacteria""",1330,gacgaacgctggcggcgtgcctaacacatgcaagtcgtacgcggtg...,Acidimicrobidae,"""Acidimicrobineae"""
27,Actinobacteria,Bacteria,Acidimicrobiaceae,Acidimicrobium,Acidimicrobiales,"""Actinobacteria""",1349,gcttgccaccgagtggcgaacgggtgcgtaacacgtgaggaaccca...,Acidimicrobidae,"""Acidimicrobineae"""


In [61]:
print(df_flen['phylum'].value_counts())

"Actinobacteria"               160837
"Proteobacteria"               150904
Firmicutes                     131742
"Bacteroidetes"                 54214
Cyanobacteria/Chloroplast       11345
"Chloroflexi"                    8599
"Fusobacteria"                   6287
"Planctomycetes"                 4620
"Verrucomicrobia"                2697
"Acidobacteria"                  2377
"Spirochaetes"                   2337
Candidatus Saccharibacteria      1767
"Tenericutes"                    1527
"Lentisphaerae"                   895
"Deinococcus-Thermus"             694
"Gemmatimonadetes"                571
SR1                               444
Nitrospirae                       421
"Armatimonadetes"                 315
Nitrospinae                       295
"Thermotogae"                     274
"Aquificae"                       235
"Synergistetes"                   233
Cloacimonetes                     211
"Fibrobacteres"                   168
Aminicenantes                     155
candidate di

In [63]:
print(df_flen['class'].value_counts())

Actinobacteria           160757
Clostridia                64433
Gammaproteobacteria       64158
Bacilli                   63387
Betaproteobacteria        41156
Alphaproteobacteria       38602
"Bacteroidia"             29772
Flavobacteriia            16728
Cyanobacteria              8032
Fusobacteriia              6287
Anaerolineae               5152
Sphingobacteriia           4143
Deltaproteobacteria        3910
Chloroplast                3291
Erysipelotrichia           2602
Planctomycetia             2466
Epsilonproteobacteria      2341
Spirochaetia               2337
Cytophagia                 1820
Verrucomicrobiae           1600
Mollicutes                 1527
Phycisphaerae              1156
Deinococci                  694
Acidobacteria_Gp1           644
Oligosphaeria               609
Gemmatimonadetes            571
Acidobacteria_Gp3           541
Negativicutes               526
Chloroflexia                517
Acidobacteria_Gp4           448
                          ...  
Deferrib

In [64]:
print(df_flen['order'].value_counts())

Actinomycetales                       154934
Clostridiales                          63886
Lactobacillales                        52934
Burkholderiales                        29893
"Bacteroidales"                        29772
Pseudomonadales                        23872
"Enterobacteriales"                    17671
"Flavobacteriales"                     16728
Rhizobiales                            12529
Bacillales                             10404
Pasteurellales                          9713
Neisseriales                            8693
Rhodobacterales                         7992
"Fusobacteriales"                       6287
Sphingomonadales                        5415
Anaerolineales                          5152
"Sphingobacteriales"                    4143
Rhodospirillales                        3522
Caulobacterales                         2787
Erysipelotrichales                      2602
Xanthomonadales                         2356
Spirochaetales                          2337
Planctomyc

In [65]:
print(df_flen['family'].value_counts())

Corynebacteriaceae                  62278
Propionibacteriaceae                57371
Streptococcaceae                    41501
Lachnospiraceae                     26899
Enterobacteriaceae                  17671
Moraxellaceae                       16889
Ruminococcaceae                     16340
Flavobacteriaceae                   15978
Comamonadaceae                      13734
Pasteurellaceae                      9713
Micrococcaceae                       8739
Neisseriaceae                        8693
Rhodobacteraceae                     7992
"Porphyromonadaceae"                 7800
Bacteroidaceae                       7785
"Prevotellaceae"                     7484
Clostridiales_Incertae Sedis XI      7278
Burkholderiales_incertae_sedis       7247
Pseudomonadaceae                     6970
Anaerolineaceae                      5152
Burkholderiaceae                     5121
Sphingomonadaceae                    4938
Bacillaceae 1                        4559
"Fusobacteriaceae"                

In [66]:
print(df_flen['genus'].value_counts())

Corynebacterium                  61768
Propionibacterium                56755
Streptococcus                    40628
Cloacibacterium                   8676
Bacteroides                       7780
Aquabacterium                     7025
Diaphorobacter                    6891
Moraxella                         6719
Acinetobacter                     6696
Pseudomonas                       6603
Serratia                          5912
Prevotella                        5788
Faecalibacterium                  5217
Neisseria                         5182
Haemophilus                       4380
Anaerococcus                      4361
Acidovorax                        4227
Bacillus                          4186
Rothia                            4186
Porphyromonas                     3565
Fusobacterium                     3462
Lachnospiracea_incertae_sedis     3443
Dolosigranulum                    3435
Candidatus Pelagibacter           3368
Aerococcus                        3216
Burkholderia             